In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
import numpy as np
import time

In [86]:
def Time(s):
    str = time.asctime(time.localtime())+". At %d s, " % (time.time() - start) + s + '\n'
    print(str,end='')
    return str
start = time.time()
Time("xx")

Fri May  3 18:07:45 2019. At 0 s, wtf


'Fri May  3 18:07:45 2019. At 0 s, wtf\n'

## An example of TF-IDF

In [ ]:
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

print(X.shape)

In [6]:
print(X.toarray())
X[1].toarray()

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.6876236  0.         0.28108867 0.         0.53864762
  0.28108867 0.         0.28108867]
 [0.51184851 0.         0.         0.26710379 0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


array([[0.        , 0.6876236 , 0.        , 0.28108867, 0.        ,
        0.53864762, 0.28108867, 0.        , 0.28108867]])

In [11]:
a, b =corpus[1].split(' ',1)
a

'This'

## Getting feature matrix of TF-IDF

In [102]:
def get_tf_idf_feature_matrix(corpus, combining, use_idf = True):
    documents = []
    le = preprocessing.LabelEncoder()
    if combining == "avg":
        with open("data/%s/%s-train-stemmed.txt" % (corpus, corpus),'r') as f:
            collection = f.readlines()
            documents = []
            n = len(collection)
            y_train = [""]*n
            for i, a_line in enumerate(collection):
                words = a_line.split()
                y_train[i] = words[0]
                doc = ' '.join(words[1:])
                '''
                split = a_line.split(' ',1)
                if (len(split)==2):
                    y_train[i]= split[0]
                    doc = split[1]
                else:
                    y_train[i] = split[0]
                    doc = ""
                '''
                documents.append(doc)
            y_train = le.fit_transform(y_train)
            vectorizer = TfidfVectorizer(use_idf = use_idf, stop_words = None)
            x_train = vectorizer.fit_transform(documents)
            print(x_train.shape)
        with open("data/%s/%s-test-stemmed.txt" % (corpus, corpus),'r') as f:
            collection = f.readlines()
            documents = []
            n = len(collection)
            y_test = [""]*n
            for i, a_line in enumerate(collection):
                words = a_line.split()
                y_test[i] = words[0]
                doc = ' '.join(words[1:])
                documents.append(doc)
            x_test = vectorizer.transform(documents)
            y_test = le.transform(y_test)
#         x_train[np.isnan(x_train)] = 0
#         x_test[np.isnan(x_test)] = 0
        return x_train, y_train, x_test, y_test, le.classes_

## Baseline for webKB

### TF

In [105]:
corpus = "webkb"
X_train, y_train, X_test, y_test, categories = get_tf_idf_feature_matrix(corpus, "avg", False)

(2803, 7287)


In [107]:
from sklearn import svm, metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
svc = svm.LinearSVC()
parameters = [{'C': [0.01, 0.1, 1, 10, 100, 1000]}]
clf = GridSearchCV(svc, parameters, n_jobs=-1, cv=10)
print("Training the classifier...")
clf.fit(X_train,y_train)
forest = clf.fit(X_train, y_train)
report = ""
pred_train = forest.predict(X_train)

# training score
score = accuracy_score(y_train, pred_train)
# score = metrics.f1_score(y_test, pred_test, pos_label=list(set(y_test)))
acc = "Accuracy in training set:" + str(score)
mac = "Macro:" + str(metrics.precision_recall_fscore_support(y_train, pred_train, average='macro'))
mic = "Micro:" + str(metrics.precision_recall_fscore_support(y_train, pred_train, average='micro'))
met = metrics.classification_report(y_train, pred_train, target_names=categories, digits=4)

report += "\nFeatures shape:" + str(X_train.shape) + "\n"
report += '\n'.join([acc, mac, mic, met])

pred_test = forest.predict(X_test)

# testing score
score = accuracy_score(y_test, pred_test)
# score = metrics.f1_score(y_test, pred_test, pos_label=list(set(y_test)))
acc = "Accuracy in testing set:" + str(score)
mac = "Macro test:" + str(metrics.precision_recall_fscore_support(y_test, pred_test, average='macro'))
mic = "Micro test:" + str(metrics.precision_recall_fscore_support(y_test, pred_test, average='micro'))
met = metrics.classification_report(y_test, pred_test, target_names=categories, digits=4)
report += '\n'+'\n'.join([acc, mac, mic, met])
report += Time("all done.")

print(report)

Training the classifier...
Tue May  7 16:26:38 2019. At 339532 s, all done.

Features shape:(2803, 7287)
Accuracy in training set:0.9864430966821263
Macro:(0.9880176414276283, 0.9840598618001616, 0.9859613664862547, None)
Micro:(0.9864430966821263, 0.9864430966821263, 0.9864430966821263, None)
              precision    recall  f1-score   support

      course     0.9825    0.9952    0.9888       620
     faculty     0.9946    0.9733    0.9838       750
     project     0.9939    0.9732    0.9835       336
     student     0.9811    0.9945    0.9878      1097

   micro avg     0.9864    0.9864    0.9864      2803
   macro avg     0.9880    0.9841    0.9860      2803
weighted avg     0.9865    0.9864    0.9864      2803

Accuracy in testing set:0.9097421203438395
Macro test:(0.90284653330314, 0.8992648085627213, 0.9009930564226343, None)
Micro test:(0.9097421203438395, 0.9097421203438395, 0.9097421203438396, None)
              precision    recall  f1-score   support

      course     0

### TF-IDF

In [92]:
corpus = "webkb"
X_train, y_train, X_test, y_test, categories = get_tf_idf_feature_matrix(corpus, "avg")

(2803, 7287)


In [93]:
from sklearn import svm, metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
svc = svm.LinearSVC()
parameters = [{'C': [0.01, 0.1, 1, 10, 100, 1000]}]
clf = GridSearchCV(svc, parameters, n_jobs=-1, cv=10)
print("Training the classifier...")
clf.fit(X_train,y_train)
forest = clf.fit(X_train, y_train)
report = ""
pred_train = forest.predict(X_train)

# training score
score = accuracy_score(y_train, pred_train)
# score = metrics.f1_score(y_test, pred_test, pos_label=list(set(y_test)))
acc = "Accuracy in training set:" + str(score)
mac = "Macro:" + str(metrics.precision_recall_fscore_support(y_train, pred_train, average='macro'))
mic = "Micro:" + str(metrics.precision_recall_fscore_support(y_train, pred_train, average='micro'))
met = metrics.classification_report(y_train, pred_train, target_names=categories, digits=4)

report += "\nFeatures shape:" + str(X_train.shape) + "\n"
report += '\n'.join([acc, mac, mic, met])

pred_test = forest.predict(X_test)

# testing score
score = accuracy_score(y_test, pred_test)
# score = metrics.f1_score(y_test, pred_test, pos_label=list(set(y_test)))
acc = "Accuracy in testing set:" + str(score)
mac = "Macro test:" + str(metrics.precision_recall_fscore_support(y_test, pred_test, average='macro'))
mic = "Micro test:" + str(metrics.precision_recall_fscore_support(y_test, pred_test, average='micro'))
met = metrics.classification_report(y_test, pred_test, target_names=categories, digits=4)
report += '\n'+'\n'.join([acc, mac, mic, met])
report += Time("all done.")

print(report)

Training the classifier...
Fri May  3 18:18:53 2019. At 668 s, all done.

Features shape:(2803, 7287)
Accuracy in training set:0.9946485907955762
Macro:(0.9953956947920779, 0.9939787352953944, 0.9946613396980591, None)
Micro:(0.9946485907955762, 0.9946485907955762, 0.9946485907955762, None)
              precision    recall  f1-score   support

      course     0.9888    1.0000    0.9944       620
     faculty     1.0000    0.9867    0.9933       750
     project     1.0000    0.9911    0.9955       336
     student     0.9927    0.9982    0.9955      1097

   micro avg     0.9946    0.9946    0.9946      2803
   macro avg     0.9954    0.9940    0.9947      2803
weighted avg     0.9947    0.9946    0.9946      2803

Accuracy in testing set:0.8982808022922636
Macro test:(0.8936884490291144, 0.8806370238711321, 0.8866196973985003, None)
Micro test:(0.8982808022922636, 0.8982808022922636, 0.8982808022922636, None)
              precision    recall  f1-score   support

      course     0.

### TF-IDF + TCNE

In [120]:
corpus = "webkb"
X_train, y_train, X_test, y_test, categories = get_tf_idf_feature_matrix(corpus, "avg")

(2803, 7287)


In [121]:
from src.feature_generation import get_feature_matrix
from src.build_graph import build_col_graph_from_train
from src.graph_to_embedding import graph_to_embedding
from src.graph import Graph
combining = 'avg'
corpus = "webkb"
method = 'node2vec'
p = 0.3
q = 1
emb_dim = 128
graph_filepath = build_col_graph_from_train(corpus)
g = Graph()
g.read_edgelist(filename=graph_filepath, weighted=True, directed=False)
emb = graph_to_embedding(g, method, corpus, emb_dim, p, q)
X_train_1, y_train, X_test_1, y_test, categories = get_feature_matrix(emb, corpus, combining)


/home/sunxiaoyu/PycharmProjects/TCNE
Reading...
Name: 
Type: DiGraph
Number of nodes: 7287
Number of edges: 660051
Average in degree:  90.5793
Average out degree:  90.5793
Embedding node2vec read from fle.


/home/sunxiaoyu/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/sunxiaoyu/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [127]:
print(X_train_1.shape)
print(X_test_1.shape)
print(X_train.shape)
print(X_test.shape)

(2803, 128)
(1396, 128)
(2803, 7287)
(1396, 7287)


In [129]:
np.concatenate((X_train, X_train_1),axis=1)

ValueError: zero-dimensional arrays cannot be concatenated

In [130]:
X_test = np.concatenate((X_test, X_test_1))

ValueError: zero-dimensional arrays cannot be concatenated

In [ ]:
from sklearn import svm, metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
svc = svm.LinearSVC()
parameters = [{'C': [0.01, 0.1, 1, 10, 100, 1000]}]
clf = GridSearchCV(svc, parameters, n_jobs=-1, cv=10)
print("Training the classifier...")
clf.fit(X_train,y_train)
forest = clf.fit(X_train, y_train)
report = ""
pred_train = forest.predict(X_train)

# training score
score = accuracy_score(y_train, pred_train)
# score = metrics.f1_score(y_test, pred_test, pos_label=list(set(y_test)))
acc = "Accuracy in training set:" + str(score)
mac = "Macro:" + str(metrics.precision_recall_fscore_support(y_train, pred_train, average='macro'))
mic = "Micro:" + str(metrics.precision_recall_fscore_support(y_train, pred_train, average='micro'))
met = metrics.classification_report(y_train, pred_train, target_names=categories, digits=4)

report += "\nFeatures shape:" + str(X_train.shape) + "\n"
report += '\n'.join([acc, mac, mic, met])

pred_test = forest.predict(X_test)

# testing score
score = accuracy_score(y_test, pred_test)
# score = metrics.f1_score(y_test, pred_test, pos_label=list(set(y_test)))
acc = "Accuracy in testing set:" + str(score)
mac = "Macro test:" + str(metrics.precision_recall_fscore_support(y_test, pred_test, average='macro'))
mic = "Micro test:" + str(metrics.precision_recall_fscore_support(y_test, pred_test, average='micro'))
met = metrics.classification_report(y_test, pred_test, target_names=categories, digits=4)
report += '\n'+'\n'.join([acc, mac, mic, met])
report += Time("all done.")

print(report)

## Baseline for 20NG

### TF

In [103]:
corpus = "20NG"
X_train, y_train, X_test, y_test, categories = get_tf_idf_feature_matrix(corpus, "avg", False)

(11293, 54577)


In [104]:
svc = svm.LinearSVC()
parameters = [{'C': [0.01, 0.1, 1, 10, 100, 1000]}]
clf = GridSearchCV(svc, parameters, n_jobs=-1, cv=10)
print("Training the classifier...")
clf.fit(X_train,y_train)
forest = clf.fit(X_train, y_train)
report = ""
pred_train = forest.predict(X_train)

# training score
score = accuracy_score(y_train, pred_train)
# score = metrics.f1_score(y_test, pred_test, pos_label=list(set(y_test)))
acc = "Accuracy in training set:" + str(score)
mac = "Macro:" + str(metrics.precision_recall_fscore_support(y_train, pred_train, average='macro'))
mic = "Micro:" + str(metrics.precision_recall_fscore_support(y_train, pred_train, average='micro'))
met = metrics.classification_report(y_train, pred_train, target_names=categories, digits=4)

report += "\nFeatures shape:" + str(X_train.shape) + "\n"
report += '\n'.join([acc, mac, mic, met])

pred_test = forest.predict(X_test)

# testing score
score = accuracy_score(y_test, pred_test)
# score = metrics.f1_score(y_test, pred_test, pos_label=list(set(y_test)))
acc = "Accuracy in testing set:" + str(score)
mac = "Macro test:" + str(metrics.precision_recall_fscore_support(y_test, pred_test, average='macro'))
mic = "Micro test:" + str(metrics.precision_recall_fscore_support(y_test, pred_test, average='micro'))
met = metrics.classification_report(y_test, pred_test, target_names=categories, digits=4)
report += '\n'+'\n'.join([acc, mac, mic, met])
report += Time("all done.")

print(report)

Training the classifier...
Sat May  4 18:03:41 2019. At 86155 s, all done.

Features shape:(11293, 54577)
Accuracy in training set:0.9936243690781901
Macro:(0.9937145607536706, 0.9934610435105654, 0.9935771698132836, None)
Micro:(0.9936243690781901, 0.9936243690781901, 0.9936243690781901, None)
                          precision    recall  f1-score   support

             alt.atheism     0.9979    0.9896    0.9937       480
           comp.graphics     0.9948    0.9897    0.9923       584
 comp.os.ms-windows.misc     0.9843    0.9878    0.9860       572
comp.sys.ibm.pc.hardware     0.9762    0.9729    0.9745       590
   comp.sys.mac.hardware     0.9931    0.9896    0.9913       578
          comp.windows.x     0.9949    0.9966    0.9958       593
            misc.forsale     0.9681    0.9863    0.9771       585
               rec.autos     0.9966    0.9949    0.9958       594
         rec.motorcycles     1.0000    0.9950    0.9975       598
      rec.sport.baseball     1.0000    1.00

### TF_IDF

In [100]:
corpus = "20NG"
X_train, y_train, X_test, y_test, categories = get_tf_idf_feature_matrix(corpus, "avg")

(11293, 54577)


In [101]:
svc = svm.LinearSVC()
parameters = [{'C': [0.01, 0.1, 1, 10, 100, 1000]}]
clf = GridSearchCV(svc, parameters, n_jobs=-1, cv=10)
print("Training the classifier...")
clf.fit(X_train,y_train)
forest = clf.fit(X_train, y_train)
report = ""
pred_train = forest.predict(X_train)

# training score
score = accuracy_score(y_train, pred_train)
# score = metrics.f1_score(y_test, pred_test, pos_label=list(set(y_test)))
acc = "Accuracy in training set:" + str(score)
mac = "Macro:" + str(metrics.precision_recall_fscore_support(y_train, pred_train, average='macro'))
mic = "Micro:" + str(metrics.precision_recall_fscore_support(y_train, pred_train, average='micro'))
met = metrics.classification_report(y_train, pred_train, target_names=categories, digits=4)

report += "\nFeatures shape:" + str(X_train.shape) + "\n"
report += '\n'.join([acc, mac, mic, met])

pred_test = forest.predict(X_test)

# testing score
score = accuracy_score(y_test, pred_test)
# score = metrics.f1_score(y_test, pred_test, pos_label=list(set(y_test)))
acc = "Accuracy in testing set:" + str(score)
mac = "Macro test:" + str(metrics.precision_recall_fscore_support(y_test, pred_test, average='macro'))
mic = "Micro test:" + str(metrics.precision_recall_fscore_support(y_test, pred_test, average='micro'))
met = metrics.classification_report(y_test, pred_test, target_names=categories, digits=4)
report += '\n'+'\n'.join([acc, mac, mic, met])
report += Time("all done.")

print(report)

Training the classifier...
Sat May  4 17:29:26 2019. At 84101 s, all done.

Features shape:(11293, 54577)
Accuracy in training set:0.9982289914106084
Macro:(0.9983048253837111, 0.998260848687322, 0.9982773536223088, None)
Micro:(0.9982289914106084, 0.9982289914106084, 0.9982289914106084, None)
                          precision    recall  f1-score   support

             alt.atheism     1.0000    0.9979    0.9990       480
           comp.graphics     0.9983    0.9932    0.9957       584
 comp.os.ms-windows.misc     0.9983    1.0000    0.9991       572
comp.sys.ibm.pc.hardware     0.9916    0.9949    0.9932       590
   comp.sys.mac.hardware     1.0000    0.9965    0.9983       578
          comp.windows.x     1.0000    1.0000    1.0000       593
            misc.forsale     0.9832    1.0000    0.9915       585
               rec.autos     0.9983    0.9983    0.9983       594
         rec.motorcycles     1.0000    0.9967    0.9983       598
      rec.sport.baseball     1.0000    1.000

## Dustbin

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents)
print(vectorizer.get_feature_names())

print(X.shape)

In [48]:
corpus = "webkb"
new_collection = []
with open("data/%s/%s-train-stemmed.txt" % (corpus, corpus),'r') as f:
    collection = f.readlines()
    for doc in collection:
        words = doc.split()
        if len(words) > 1:
            doc = ' '.join(words)+'\n'
            new_collection.append(doc)
with open("data/%s/%s-train-stemmed.txt" % (corpus, corpus),'w') as f:
    f.writelines(new_collection)

In [55]:
len(new_collection)

2785